In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

In [2]:
# Set up the Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
# Open the webpage
driver.get("https://vahan.parivahan.gov.in/vahan4dashboard/vahan/vahan/view/reportview.xhtml")

# Wait for the page to load completely
driver.implicitly_wait(10)

In [6]:
# Function to click on an element using JavaScript
def click_with_js(driver, by, value):
    element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((by, value)))
    driver.execute_script("arguments[0].scrollIntoView(true);", element)  # Scroll to the element
    driver.execute_script("arguments[0].click();", element)

# List of specified vehicle classes to extract data for
target_vehicle_classes = [
    "ADAPTED VEHICLE",
    "MOTOR CAB",
    "MOTOR CAR",
    "OMNI BUS (PRIVATE USE)",
    "PRIVATE SERVICE VEHICLE (INDIVIDUAL USE)",
    "QUADRICYCLE (PRIVATE)"
]

# Path to save the CSV file
save_path = r"C:\Users\DELL\Downloads\vehicle_data_5.csv"

# Prepare to save the data in a CSV file
with open(save_path, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['Vehicle Class', 'Year', 'S.No', 'Vehicle_Class','JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'TOTAL'])

    # Loop through each year
    for year in range(2001, 2025):
        # Set dropdown values for each year
        click_with_js(driver, By.ID, "j_idt25_label")  # Type: Actual Value
        type_option = driver.find_element(By.XPATH, "//li[@data-label='Actual Value']")
        driver.execute_script("arguments[0].click();", type_option)
        time.sleep(2)

        click_with_js(driver, By.ID, "j_idt34_label")  # State: All Vahan4 Running States (35/36)
        state_option = driver.find_element(By.XPATH, "//li[@data-label='All Vahan4 Running States (35/36)']")
        driver.execute_script("arguments[0].click();", state_option)
        time.sleep(2)

        click_with_js(driver, By.ID, "selectedRto_label")  # RTO: All Vahan4 Running Office (1362/1434)
        rto_option = driver.find_element(By.XPATH, "//li[@data-label='All Vahan4 Running Office(1362/1434)']")
        driver.execute_script("arguments[0].click();", rto_option)
        time.sleep(2)

        click_with_js(driver, By.ID, "yaxisVar_label")  # Y-Axis: Vehicle Class
        yaxis_option = driver.find_element(By.XPATH, "//li[@data-label='Vehicle Class']")
        driver.execute_script("arguments[0].click();", yaxis_option)
        time.sleep(2)

        click_with_js(driver, By.ID, "xaxisVar_label")  # X-Axis: Month Wise
        xaxis_option = driver.find_element(By.XPATH, "//li[@data-label='Month Wise']")
        driver.execute_script("arguments[0].click();", xaxis_option)
        time.sleep(2)

        click_with_js(driver, By.ID, "selectedYearType_label")  # Year Type: Calendar Year
        year_type_option = driver.find_element(By.XPATH, "//li[@data-label='Calendar Year']")
        driver.execute_script("arguments[0].click();", year_type_option)
        time.sleep(2)

        click_with_js(driver, By.ID, "selectedYear_label")  # Year: 2024
        year_option = driver.find_element(By.XPATH, f"//li[@data-label='{year}']")
        driver.execute_script("arguments[0].click();", year_option)
        time.sleep(5)

        # Click the Refresh Button to update the table
        click_with_js(driver, By.XPATH, "//span[text()='Refresh']")
        time.sleep(10)

        page_number = 1
        found_vehicle_classes = set()

        while page_number <= 3:
            # Extract data from the table
            table_rows = driver.find_elements(By.XPATH, "//tbody[@id='groupingTable_data']//tr")

            # Loop through each row in the table
            for row in table_rows:
                vehicle_class = row.find_element(By.XPATH, ".//label[contains(@id, 'j_idt110:1:j_idt112')]").text
                if vehicle_class in target_vehicle_classes and vehicle_class not in found_vehicle_classes:
                    data = [vehicle_class, year]  # Add year to the data
                    month_data = row.find_elements(By.XPATH, ".//td[@role='gridcell']")
                    for cell in month_data:
                        data.append(cell.text.strip())
                    # Write the row data to the CSV file
                    writer.writerow(data)
                    found_vehicle_classes.add(vehicle_class)

            # Move to the next page if it exists
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//span[@class='ui-icon ui-icon-seek-next']"))
                )
                driver.execute_script("arguments[0].click();", next_button)
                # Wait for the next page to load
                time.sleep(5)
                page_number += 1
            
            except Exception as e:
                print("No more pages or an error occurred:", e)
                break

In [7]:
# Close the browser
driver.quit()